In [ ]:
# Librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# 1️⃣ Exploración del Dataset (EDA)

# Cargar el dataset
file_path = 'twittersentimentanalysis.csv'  # Cambia esta ruta
data = pd.read_csv(file_path)

# Inspección inicial
print("Primeras filas del dataset:")
print(data.head())
print("\nDistribución de sentimientos:")
print(data['Sentiment'].value_counts())

# Visualización de la distribución de sentimientos
plt.figure(figsize=(8, 6))
sns.countplot(data=data, x='Sentiment', palette='viridis')
plt.title('Distribución de Sentimientos')
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad')
plt.show()

# Generación de WordCloud
def generar_wordcloud(sentimiento):
    texto = " ".join(data[data['Sentiment'] == sentimiento]['Tweet'].astype(str))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Nube de Palabras para Sentimiento {sentimiento}')
    plt.show()

generar_wordcloud('positive')
generar_wordcloud('negative')
generar_wordcloud('neutral')

# 2️⃣ Preprocesamiento y Entrenamiento del Modelo

# Configuración del preprocesamiento
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

def preprocess(texto):
    tokens = word_tokenize(texto.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

data['texto_procesado'] = data['Tweet'].apply(preprocess)

# Vectorización y división de datos
X = data['texto_procesado']
y = data['Sentiment']

vectorizer = TfidfVectorizer(max_features=5000)
X_vectorizado = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vectorizado, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Evaluación del modelo
y_pred = modelo.predict(X_test)
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

# 3️⃣ Guardar el modelo y el vectorizador
with open('modelo.pkl', 'wb') as archivo_modelo:
    pickle.dump(modelo, archivo_modelo)
with open('vectorizador.pkl', 'wb') as archivo_vectorizador:
    pickle.dump(vectorizer, archivo_vectorizador)

print("\n¡El modelo y el vectorizador han sido guardados exitosamente!")


Cargando datos...


FileNotFoundError: [Errno 2] No such file or directory: './data'